In [1]:
import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import h5py
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]

vmec_file = vmec_files[1]
import StellBlanket.SBGeom as SBGeom
from functools import partial
from jax_sbgeom.flux_surfaces.flux_surfaces_base import ToroidalExtent, _cylindrical_position_interpolated, _interpolate_array
from jax_sbgeom.flux_surfaces.flux_surfaces_extended import FluxSurfaceNormalExtended
#from jax_sbgeom.flux_surfaces.flux_surface_meshing import _mesh_watertight_surfaces_connectivity
from tests.flux_surfaces.test_flux_surface_base import test_position, _mesh_to_pyvista_mesh, test_watertight_surfaces, _check_vectorized_internal, time_jsb_function, print_timings
import jax_sbgeom.flux_surfaces.flux_surface_meshing as fsm
import jax_sbgeom.flux_surfaces.flux_surfaces_base as fsb
import jax_sbgeom.flux_surfaces.flux_surfaces_extended as fse
import pyvista as pv
from dataclasses import dataclass

Triangle Elements not compiled


In [2]:
def _get_flux_surfaces(vmec_file):
    fs_jax    = jsb.flux_surfaces.FluxSurface.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file)
    return fs_jax, fs_sbgeom

def _get_extended_flux_surfaces(vmec_file):    
    fs_jax    = jsb.flux_surfaces.FluxSurfaceNormalExtended.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.Flux_Surfaces_Normal_Extended(SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file))
    return fs_jax, fs_sbgeom

def _get_extended_no_phi_flux_surfaces(vmec_file):    
    fs_jax    = jsb.flux_surfaces.FluxSurfaceNormalExtendedNoPhi.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.Flux_Surfaces_Normal_Extended_No_Phi(SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file))
    return fs_jax, fs_sbgeom

def _get_extended_constant_phi_flux_surfaces(vmec_file):    
    fs_jax    = jsb.flux_surfaces.FluxSurfaceNormalExtendedConstantPhi.from_hdf5(vmec_file)
    fs_sbgeom = SBGeom.Flux_Surfaces_Normal_Extended_Constant_Phi(SBGeom.VMEC.Flux_Surfaces_From_HDF5(vmec_file))
    return fs_jax, fs_sbgeom

fs_jax, fs_sbgeom = _get_extended_constant_phi_flux_surfaces(vmec_file)  # just to compile

fs_np , _ = _get_extended_no_phi_flux_surfaces(vmec_file)  # just to compile

In [3]:
s = 1.0
d = 0.5
theta = 0.2 
phi = 0.3
posj = fse._normal_extended_constant_phi_cartesian_position(fs_jax.data, fs_jax.settings, s+d, theta, phi)
posb = fs_sbgeom.Return_Position(s, d, theta, phi)


print(jnp.arctan2(posj[1], posj[0]))
print(jnp.arctan2(posb[1], posb[0]))
print(posj, posb)

0.3
0.30000000176831865
[23.25389392  7.19327233  0.86935654] [23.2538939   7.19327237  0.86935654]


In [4]:
def _check_normals_both(fs_jax, fs_sbgeom, sampling_func, sampling_func_1d, n_repetitions=1, atol =1e-13):
    norm_jax, time_jax, std_jax          = time_jsb_function(fs_jax.normal, *sampling_func(fs_jax, include_axis=False),              n_repetitions= n_repetitions)
    norm_sbgeom, time_sbgeom, std_sbgeom = time_jsb_function(fs_sbgeom.Return_Normal, *sampling_func_1d(fs_jax, include_axis=False, reverse_theta=fs_sbgeom.du_x_dv_sign() == 1.0), n_repetitions= n_repetitions, jsb=False)    
    print(norm_sbgeom)
    assert jnp.allclose(norm_jax, norm_sbgeom.reshape(norm_jax.shape), atol= atol)
    print_timings("Normal", time_jax, std_jax, time_sbgeom, std_sbgeom)

from tests.flux_surfaces.test_flux_surface_base_extension import _extended_sampling_grid, _extended_1d_sampling_grid, _extended_1d_sampling_grid_stop_1
_check_normals_both(fs_jax, fs_sbgeom, _extended_sampling_grid, _extended_1d_sampling_grid, n_repetitions=1, atol=1e-6)

[[ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [ 8.34187588e-01  4.69597109e-01  2.89153288e-01]
 [ 8.09016994e-01  5.87785252e-01  2.33232033e-16]
 [ 7.04391532e-01  6.48246054e-01 -2.89153288e-01]
 [ 3.09016994e-01  9.51056516e-01 -6.27892743e-17]
 [ 8.56809753e-01 -2.75825063e-02  5.14894409e-01]
 [ 6.36016349e-01  4.28018263e-01  6.42093116e-01]
 [ 2.40658586e-01  3.29734205e-01  9.12884877e-01]
 [ 5.73263144e-01  4.76340726e-01  6.66684993e-01]
 [ 2.91001297e-01  8.06351036e-01  5.14894409e-01]
 [-5.25337423e-01 -2.78216771e-01  8.04124381e-01]
 [ 1.52634970e-02  1.03084958e-01  9.94555437e-01]
 [-1.41002913e-01  2.71701548e-01  9.51996033e-01]
 [-4.41859277e-01 -4.47116039e-01  7.77719504e-01]
 [ 1.02261682e-01 -5.85599290e-01  8.04124381e-01]
 [-9.95610857e-01 -7.90296955e-02 -5.01331130e-02]
 [-7.99101703e-01 -5.74635597e-01  1.76721246e-01]
 [-7.96753102e-01 -3.32569965e-01  5.04560911e-01]
 [-6.60176368e-01 -4.10745963e-01  6.28852063e-01]
 [-2.32498968e-01 -9.71303712e-

In [5]:
print(fs_jax.cartesian_position(s+d, theta, phi))
print(fs_sbgeom.Return_Position(s, d, theta, phi))

print(fs_jax.principal_curvatures(s+d, theta, phi))
print(fs_sbgeom.Return_Principal_Curvatures(s, d, theta, phi))

print(fs_jax.normal(s+d, theta, phi))


print(fs_sbgeom.Return_Normal(s, d, theta, phi))

[23.25389392  7.19327233  0.86935654]
[23.2538939   7.19327237  0.86935654]
[0.00846512 0.20085576]
[0.00846512 0.20085576]
[0.84046615 0.4460473  0.30766614]
[0.84046615 0.4460473  0.30766614]


In [6]:
%timeit fs_np.cartesian_position(1.5, 0.2, jnp.linspace(0, 2*jnp.pi, 10000000)).block_until_ready()

1.44 s ± 9.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
10000000 / 500000

20.0

In [9]:
%timeit fs_jax.cartesian_position(1.5, 0.2, jnp.linspace(0, 2*jnp.pi, 500000)).block_until_ready()

579 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
_check_vectorized_internal(fs_jax.cartesian_position)

In [52]:
fs_jax.cartesian_position(1.5, jnp.array([0.1, 0.2]), jnp.array([0.3, 0.4]))

Array([[23.33713165,  7.21902078,  0.57565824],
       [22.14944684,  9.36463592,  0.63932094]], dtype=float64)

In [53]:
fs_sbgeom.Return_Position(0.5 * onp.ones(2), 0.0 * onp.ones(2), onp.array([0.1, 0.2]), onp.array([0.3, 0.4]))

array([[22.60702489,  6.99317229,  0.69869452],
       [21.40116221,  9.04826626,  0.6622971 ]])

In [38]:
jnp.arctan2(posj[1], posj[0]), onp.arctan2(posb[1], posb[0])

(Array(0.3, dtype=float64), np.float64(0.30000000176831865))

In [23]:
fs_sbgeom.Return_Position(1.0, 0.45, 0.1, 0.2)

array([24.25767128,  4.91727343,  0.66072459])

In [11]:
s = 1.0
d = 0.5
theta = 0.2 
phi = 0.3
print(fse._normal_extended_no_phi_cartesian_position(fs_jax.data, fs_jax.settings, s+d, theta, phi))
fs_sbgeom.Return_Position(s, d, theta, phi)

[-0.29552021  0.95533649  0.        ]
0.17896749729128478
[0.8390241  0.44687508 0.31038882] [0.90654885 0.28042842 0.31548229]
[23.24725547  7.19121882  0.86441375]


array([23.24725547,  7.19121882,  0.86441375])

In [5]:
fs_sbgeom.Return_Position(1.0, 0.0, 0.2,0.3)

array([22.79398104,  7.05100461,  0.7066726 ])

In [6]:
ss, tt = jnp.meshgrid(jnp.array([1.5, 1.6]), jnp.linspace(0, 2 * jnp.pi, 5), indexing='ij')
fs_jax.cartesian_position(1.5, ss, tt)

JitTracer<float64[2,5,3]>
JitTracer<float64[2,5]>
JitTracer<float64[2,5,3]> JitTracer<float64[2,5,3]>


Array([[[ 2.33435006e+01,  0.00000000e+00,  3.54410065e+00],
        [ 1.34279273e-15,  2.19181387e+01,  3.75767817e+00],
        [-2.14677661e+01,  2.63225847e-15,  1.77876625e+00],
        [-4.25162392e-15, -2.31311040e+01,  1.65641767e+00],
        [ 2.33435006e+01, -5.71979257e-15,  3.54410065e+00]],

       [[ 2.30718270e+01,  0.00000000e+00,  3.60813365e+00],
        [ 1.33193695e-15,  2.16642790e+01,  3.85756590e+00],
        [-2.12250909e+01,  2.60256882e-15,  1.89022671e+00],
        [-4.22574040e-15, -2.30197949e+01,  1.55279433e+00],
        [ 2.30718270e+01, -5.65798786e-15,  3.60813365e+00]]],      dtype=float64)

In [7]:
from jax_sbgeom.flux_surfaces.flux_surfaces_extended import FluxSurfaceNormalExtended, _normal_extended_principal_curvatures, _normal_extended_principal_curvatures_jit
_normal_extended_principal_curvatures(fs_jax.data, fs_jax.settings, 1.5, ss, tt)

Array([[[ 0.03936199,  1.01773538],
        [ 0.05314682,  0.50113663],
        [-0.13579406,  0.06603556],
        [-0.49497958, -0.03873364],
        [ 0.03936199,  1.01773538]],

       [[ 0.02356461,  1.33404579],
        [ 0.05116669,  0.72522694],
        [-0.08582024,  0.11588139],
        [-0.4978989 , -0.04412146],
        [ 0.02356461,  1.33404579]]], dtype=float64)

In [8]:
fs_jax.cartesian_position(1.5, ss, tt)

Array([[[ 2.33435006e+01,  0.00000000e+00,  3.54410065e+00],
        [ 1.34279273e-15,  2.19181387e+01,  3.75767817e+00],
        [-2.14677661e+01,  2.63225847e-15,  1.77876625e+00],
        [-4.25162392e-15, -2.31311040e+01,  1.65641767e+00],
        [ 2.33435006e+01, -5.71979257e-15,  3.54410065e+00]],

       [[ 2.30718270e+01,  0.00000000e+00,  3.60813365e+00],
        [ 1.33193695e-15,  2.16642790e+01,  3.85756590e+00],
        [-2.12250909e+01,  2.60256882e-15,  1.89022671e+00],
        [-4.22574040e-15, -2.30197949e+01,  1.55279433e+00],
        [ 2.30718270e+01, -5.65798786e-15,  3.60813365e+00]]],      dtype=float64)

In [9]:
fs_jax.principal_curvatures(1.5, ss, tt)

Array([[[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]],

       [[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]]], dtype=float64, weak_type=True)